In [9]:
# 好きなラーメン店舗14件の店名
ramen_restaurants = [
    "ラーメン二郎目黒店",
    "元祖台湾まぜそば 麺屋はなび",
    "一幸舎",
    "中華蕎麦 とみ田",
    "壱角家",
    "一風堂",
    "蒙古タンメン中本",
    "家家家 本郷店 （YAH-YAH-YA）",
    "博多らーめん ShinShin",
    "中華そば きび",
    "千里眼",
    "ラーメン鷹の目 明大前",
    "黒亭ラーメン",
    "桂花ラーメン　太肉麺"
]

In [10]:
# ラーメンの画像をラーメン店舗ごとに検索する
from icrawler.builtin import BingImageCrawler

for restaurant in ramen_restaurants:
    keyword = restaurant + ' ラーメン'
    crawler = BingImageCrawler(
        storage = {
            'root_dir': './image/' + keyword
        }
    )
    crawler.crawl(
        keyword = keyword,
        max_num = 100,
    )

2025-04-13 16:52:58,391 - INFO - icrawler.crawler - start crawling...
2025-04-13 16:52:58,392 - INFO - icrawler.crawler - starting 1 feeder threads...
2025-04-13 16:52:58,393 - INFO - feeder - thread feeder-001 exit
2025-04-13 16:52:58,394 - INFO - icrawler.crawler - starting 1 parser threads...
2025-04-13 16:52:58,396 - INFO - icrawler.crawler - starting 1 downloader threads...
2025-04-13 16:52:58,797 - INFO - parser - parsing result page https://www.bing.com/images/async?q=ラーメン二郎目黒店 ラーメン&first=0
2025-04-13 16:52:59,122 - INFO - downloader - image #1	https://ore-ramen.com/wp/wp-content/uploads/2019/09/IMG_7534.jpg
2025-04-13 16:52:59,291 - INFO - downloader - image #2	https://ramen-scene.com/wp-content/uploads/2022/12/IMG_2426-2-1m-1-1024x768.jpg
2025-04-13 16:52:59,537 - INFO - downloader - image #3	https://tabelog.com/restaurant/images/Rvw/192144/640x640_rect_90537e777e6aae786bd2e02b19757e6e.jpg
2025-04-13 16:52:59,778 - INFO - downloader - image #4	https://ramen-jiro-pc3.com/wp-con

In [11]:
# yoloを使いclassificationで明らかに異なるものを排除する
# e.g. メニュー、冷凍食品
from ultralytics import YOLO
from pathlib import Path

model = YOLO("yolo11n-cls.pt")
probs = []
for image_path in Path('./image/').rglob('*.jpg'):
    path = str(image_path)
    results = model(path)

    result = results[0]

    # 最も確率が高いクラスのインデックスとその確率を取得
    top_idx = result.probs.top1
    top_prob = float(result.probs.top1conf.item())

    top_class = result.names[top_idx]

    probs.append((path, top_class, top_prob))

detected_objects = []
for prob in probs:
    if prob[1] in detected_objects:
        continue
    detected_objects.append(prob[1])
for obj in detected_objects:
    with open('detected_objects.txt', 'a') as f:
        f.write(f"{obj}\n")
# probsのtop_classを

# # probsの全体の数のうち、top_probが0.8以上のものである確率を表示する
# import numpy as np
# probs = np.array(probs)
# # Convert the third column to float for comparison by creating a new array for the probabilities
# float_probs = probs[:, 2].astype(float)
# probability_above_threshold = np.sum(float_probs >= 0.8) / len(float_probs) * 100
# print(f"Probability of top_prob >= 0.8: {probability_above_threshold:.2f}%")


image 1/1 /Users/irisakeishi/development-ml/ramen-classification/image/蒙古タンメン中本 ラーメン/000026.jpg: 224x224 soup_bowl 0.47, Crock_Pot 0.17, hot_pot 0.08, consomme 0.08, meat_loaf 0.05, 35.5ms
Speed: 34.0ms preprocess, 35.5ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /Users/irisakeishi/development-ml/ramen-classification/image/蒙古タンメン中本 ラーメン/000032.jpg: 224x224 vending_machine 0.57, toyshop 0.04, plunger 0.03, confectionery 0.03, bubble 0.02, 38.4ms
Speed: 10.0ms preprocess, 38.4ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /Users/irisakeishi/development-ml/ramen-classification/image/蒙古タンメン中本 ラーメン/000033.jpg: 224x224 comic_book 0.14, packet 0.13, wok 0.06, hotdog 0.06, book_jacket 0.05, 17.5ms
Speed: 11.8ms preprocess, 17.5ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /Users/irisakeishi/development-ml/ramen-classification/image/蒙古タンメン中本 ラーメン/000027.jpg: 224x224 ashcan 0.40, cocktail_shaker 0

In [12]:
# prob[2]の中で、数が多い順のvalueとその合計数をテキストファイルに保存する
import pandas as pd
df = pd.DataFrame(probs, columns=['path', 'class', 'probability'])
probability_counts = df['class'].value_counts()

with open('probability_counts.txt', 'w') as f:
    for index, value in probability_counts.items():
        f.write(f"{index}: {value}\n")


In [13]:
# probsのうちavailable_probability_countsのindexとprobs[1]が等しいprobsのpathを取得してtxtファイルに保存する
available_probability_counts = probability_counts[probability_counts.index.isin(["soup_bowl", "carbonara", "plate", "broccoli", "mashed_potato"])]
for prob in probs:
    if prob[1] in available_probability_counts.index:
        with open('available_probability_counts.txt', 'a') as f:
            f.write(f"{prob[0]}\n")

In [20]:
import random

# 再現性のためシードを固定
random.seed(42)

# 入力ファイル
input_file = "available_probability_counts.txt"

# 出力ディレクトリ（なければ作成）
training_dir = Path("training_paths")
test_dir = Path("test_paths")
training_dir.mkdir(exist_ok=True)
test_dir.mkdir(exist_ok=True)

# キーワードごとにパスをグループ化する辞書
grouped = {}

with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        # 想定されるフォーマット: "image/{keyword}/000032.jpg"
        parts = line.split("/")
        if len(parts) >= 2:
            # parts[0] が "image"、parts[1] が keyword
            keyword = parts[1]
            grouped.setdefault(keyword, []).append(line)

# 各グループで70%訓練、30%テストに分割してファイルに保存
for keyword, paths in grouped.items():
    # シャッフル
    random.shuffle(paths)
    n_train = int(len(paths) * 0.7)
    train_paths = paths[:n_train]
    test_paths  = paths[n_train:]
    
    # 訓練用、テスト用ファイルのパス
    train_file = training_dir / f"{keyword}.txt"
    test_file  = test_dir / f"{keyword}.txt"
    
    with open(train_file, "w", encoding="utf-8") as f_train:
        for p in train_paths:
            f_train.write(p.replace("image/", "image_resized/") + "\n")
    
    with open(test_file, "w", encoding="utf-8") as f_test:
        for p in test_paths:
            f_test.write(p.replace("image/", "image_resized/") + "\n")

In [ ]:
from tqdm import tqdm
from PIL import Image
# 画像サイズを全て640x640にリサイズして、編集した画像をimage_resized/に保存する
def resize_images_in_directory(input_dir, output_dir, size=(640, 640)):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    for image_path in tqdm(input_dir.rglob('*.jpg')):
        # 保存先の画像パスを構築（元のディレクトリ構造を維持）
        relative_path = image_path.relative_to(input_dir)
        new_path = output_dir / relative_path
        new_path.parent.mkdir(parents=True, exist_ok=True)
        with Image.open(image_path) as img:
            img = img.resize(size)
            img.save(new_path)

resize_images_in_directory('./image/', './image_resized/')

1015it [00:21, 47.25it/s]


In [ ]:
# training_paths内のそれぞれのtxtファイル内の一行ずつを読み込み、image_resized/内の画像を読み込み、training_image内に保存する
from pathlib import Path
import shutil
def copy_images_from_txt_to_directory(txt_dir, image_dir, output_dir):
    txt_dir = Path(txt_dir)
    image_dir = Path(image_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    for txt_file in txt_dir.rglob('*.txt'):
        with open(txt_file, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                # 画像のパスを取得
                image_path = line
                # ファイル名だけを使って、コピー先のパスを構築(サブディレクトリを無視)
                new_path = output_dir / Path(str(Path(image_path).relative_to('image_resized')).replace('/', '_'))
                new_path.parent.mkdir(parents=True, exist_ok=True)
                shutil.copy(image_path, new_path)
copy_images_from_txt_to_directory('./training_paths/', './image_resized/', './training_image/')

In [2]:
# トレーニング
from ultralytics import YOLO

model = YOLO("yolo11n-cls.pt")

results = model.train(
    data="ramen-datasets",
    epochs=16,
    imgsz=640,
    batch=32,
    save=True,
)

Ultralytics 8.3.107 🚀 Python-3.13.2 torch-2.6.0 CPU (Apple M1)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=ramen-datasets, epochs=16, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript,

train: Scanning /Users/irisakeishi/development-ml/ramen-classification/ramen-datasets/train... 374 images, 0 corrupt: 100%|██████████| 374/374 [00:00<?, ?it/s]
val: Scanning /Users/irisakeishi/development-ml/ramen-classification/ramen-datasets/test... 167 images, 0 corrupt: 100%|██████████| 167/167 [00:00<?, ?it/s]

optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000556, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/classify/train2
Starting training for 16 epochs...

      Epoch    GPU_mem       loss  Instances       Size



               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:25<00:00,  8.58s/it]

                   all      0.156      0.485


...

      16/16         0G     0.1915         22        640: 100%|██████████| 12/12 [02:47<00:00, 13.93s/it]
               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:23<00:00,  7.98s/it]

                   all      0.886      0.994



16 epochs completed in 0.967 hours.
Optimizer stripped from runs/classify/train2/weights/last.pt, 3.2MB
Optimizer stripped from runs/classify/train2/weights/best.pt, 3.2MB

Validating runs/classify/train2/weights/best.pt...
Ultralytics 8.3.107 🚀 Python-3.13.2 torch-2.6.0 CPU (Apple M1)
YOLO11n-cls summary (fused): 47 layers, 1,543,958 parameters, 0 gradients, 3.2 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /Users/irisakeishi/development-ml/ramen-classification/ramen-datasets/train... found 374 images in 14 classes ✅ 
val: /Users/irisakeishi/development-ml/ramen-classification/ramen-datasets/test... found 167 images in 14 classes ✅ 
test: /Users/irisakeishi/development-ml/ramen-classification/ramen-datasets/test... found 167 images in 14 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:22<00:00,  7.61s/it]


                   all      0.886      0.994
Speed: 0.0ms preprocess, 125.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2


In [7]:
model_path = str(results.save_dir) + "/weights/best.pt"
model = YOLO(model_path)
results_inferr = model("test_image/ラーメン鷹の目 明大前 ラーメン_000004.jpg")


image 1/1 /Users/irisakeishi/development-ml/ramen-classification/test_image/ラーメン鷹の目 明大前 ラーメン_000004.jpg: 640x640 Ramen_Taka_no_Me_Meidaimae 1.00, Senrigan 0.00, Ippudo 0.00, Ichikakuya 0.00, Ramen_Jiro_Meguro 0.00, 78.7ms
Speed: 36.0ms preprocess, 78.7ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
repo_name = "keishiiii/ramen_classifier"

to